In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import sys

sys.path.append("../../")

In [28]:
from tokenizers import Tokenizer
import sys

import matplotlib.pyplot as plt
import numpy as np
import collections
import torch

from ChEmbed.data import chembldb, smiles_dataset, chembed_tokenize
from ChEmbed.training import trainer
from ChEmbed.modules import simple_rnn

In [20]:
chembl_raw = chembldb.ChemblDB()
chembl_smiles = chembl_raw._load_or_download()["canonical_smiles"]

In [21]:
tokenizer = chembed_tokenize.load_chembed_tokenizer(filepath = "../data/tokenizers/tokenizer-chembldb-16-06-2025.json")

In [39]:
chembl_dataset = smiles_dataset.SMILESDataset(
    smiles_list = chembl_smiles,
    tokenizer = tokenizer
)

In [41]:
model = simple_rnn.simpleRNN(
    # Mandatory
    num_hiddens = 64,
    vocab_size = len(tokenizer),
    # tuning
    learning_rate = 0.1,
    weight_decay = 0.01
)

/Users/patrickrowe/miniconda3/envs/main/lib/python3.12/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [42]:
chembl_dataset[0].type(torch.float32)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [43]:
model.forward(next(iter(chembl_dataset)))

tensor([[ 0.0240, -0.0486, -0.0227,  ...,  0.1354, -0.0475, -0.1210],
        [ 0.0983, -0.0559, -0.0363,  ...,  0.1613, -0.0572, -0.0609],
        [ 0.0254, -0.1033, -0.0773,  ...,  0.1938, -0.0003,  0.0100],
        ...,
        [ 0.1143, -0.0358, -0.0784,  ...,  0.0424, -0.1109, -0.0938],
        [ 0.1143, -0.0358, -0.0784,  ...,  0.0424, -0.1109, -0.0938],
        [ 0.1143, -0.0358, -0.0784,  ...,  0.0424, -0.1109, -0.0938]],
       grad_fn=<AddmmBackward0>)

In [220]:
def predict(input, num_preds, model, dataset, device=None):
    state, outputs = None, [input[0]]

    # predict input + num_preds tokens
    for i in range(len(input) + num_preds - 1):
        print(i)
        X = torch.tensor(dataset.encode_smiles_to_one_hot(outputs[i]), device=device)
        
        rnn_outputs, state = model.rnn(X, state)

        if i < len(input) - 1:
            outputs.append(input[i + 1])
        else:
            Y = dataset.tokenizer.decode(model.linear(rnn_outputs).argmax())
            outputs.append(Y)

    return ''.join(outputs)
            

In [223]:
result = predict("COCC", 10, model, chembl_dataset)

0
1
2
3
4


/var/folders/ql/140bddls19x4s8xyk9nrwyd00000gn/T/ipykernel_16882/3921232552.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(dataset.encode_smiles_to_one_hot(outputs[i]), device=device)


5
6
7
8
9
10
11
12


In [224]:
print(result)

COCCCc1ccccc1
